# Scrapy: An advanced and powerful framework for crawling
## Introduction

In our first 688/388 homework, we had a taste on BeautifulSoup -- a nice Python library to extract data from HTML and XML files, and we have seen its power over pure python httprequest and how it saves days and hours of work. However, BeautifulSoup is just a Python library, and has many limitations like non-reusable (will be discussed later). In this tutorial, we are going to introduce a more complete framework for fetching data from the website called **Scrapy**.

## What is crawling?

To better understand the topic, we need to know what crawling is. Crawling means downloading and fetching data from the web content at one or more URLs (https://speakerdeck.com/amontalenti/web-crawling-and-metadata-extraction-in-python). like this:

In [ ]:
import requests
r = requests.get("http://example.com/foo/bar")

Or like this:

In [ ]:
import urllib2
res = urllib2.urlopen("http://www.datasciencecourse.org/tutorial.pdf").read()

Looks easy, right?
But in real projects, we have to consider a lot more than this. A slight glance of the situations/problems we must consider are as follows:
1. What is the page encoding? We can only figure that out by looking at the HTTP header and content.
2. What if the request fails?
3. Code reusability for multiple websites?

Additionally, in terms of scalability, concurrency is also a very big issue. Especially for distributed system, how to deal with mutual exclusion is always difficult for programmers.

## What is Scrapy?

Scrapy is a free and open source collaborative framework to extract data from website. it can also be used to extract data using APIs or as a general purpose web crawler. Scrapy allows user to implement the core features of the system without taking care of issues such as listed above. Scrapy also makes it easier to build and scale large crawling projects by allowing developers to re-use their code.

## Why we need a framework?

The reason that we need a framework to do the crawling is not only because of code reusability (althought this is one of the most important reasons), but there are also several other reasons as follows:
1. Scrapy framework is great for maintaining coding convention, and is human-readable and understandable among other developers.
2. Scrapy has integrated toolset which avoids from choosing specific libraries (lxml, requests, celery, redis).
3. Scrapy makes spidering web site extremely easy, and is compatible among multiple platforms.

Scrapy is a powerful framework because it is: 
1. powerful and fast. You only have to write the rules and don't have to care about the core feature of the crawling system.
2. Easily extensible. You can add many customized plugins without touching too many of its core functions, which makes Scrapy design friendly and easy to scale up.
3. Portable. Scrapy can be used on different platforms, such as Linux, OSX and Windows.

## Comparison with Other tools?
### Scrapy vs BeautifulSoup
BeautifulSoup is a Python library, and Scrapy is a complete crawling framework with integrated tools and extensible features. While Scrapy has many advantages, there are certain areas we use BeautifulSoup more.

Scrapy is a Web-spider or web scraper framework. You give Scrapy a root URL to start crawling, then you can specify constraints on how many (number of) URLs you want to crawl and fetch,etc. It is a complete framework for web-scraping or crawling. While BeautifulSoup is a parsing library which also does a pretty good job of fetching contents from URL and allows you to parse certain parts of them without any hassle. It only fetches the contents of the URL that you give and then stops. It does not crawl unless you manually put it inside an infinite loop with certain criteria.

### Scrapy vs Nutch
Apache Nutch is another frequently used web crawler implemented by java. It also has characeristics of scalable, fast, and could support distributed work (based on Hadoop) well. Another interesting point is that Scrapy is online, while Nutch is batched. It means that you have to wait until all the jobs are done to renew crawler state (schedule new batch of URLs, update scores, etc.). Scrapy allows to do all that in real-time, with predictable delay.

# Scrapy Architecture
![title](https://doc.scrapy.org/en/latest/_images/scrapy_architecture_02.png)

The data flow is clearly described in the picture above, mainly controlled by the crawl engine in the middle. The user customizes the spiders and sends request to the engine, and the engine asks for the next crawling request from the scheduler. The engine then sends the request to the downloader, waiting for the downloader to extract website data. Once finishing downloading, the data was sent throught the spiders, and after which the scraped data would be sent through the item pipelines (de-duplication, cleansing, validation, etc.).

## First Scrapy -- TechCrunch Scrawler
In this part, we will provide you with the step by step guide on how to build a scrawler to extract and process data from the  TechCrunch website using Scrapy.
### Set up new Scrapy project
If you are new to Scrapy, install Scrapy first as you do for Python packages, for example, if you are a Mac user:

In your project directory, enter the following code to set up your Scrapy projcect:

In spider folder, create your own rule class, and name it news_spider.py. The code is as follows, and the instruction for code is commented as well:

In [ ]:
import ujson

from scrapy.contrib.spiders import CrawlSpider, Rule
from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
from TechCrunch.items import TechcrunchItem


class newsFeedSpider(CrawlSpider):
    name = "newsFeed"

    allowed_domains = ["techcrunch.com"]
    start_urls = ["https://techcrunch.com"]

    rules = (
        # extract the links that you are allowed to crawl
        Rule(SgmlLinkExtractor(allow=('startups\/',))),
        # extract links matching only 2016/10/18's news and parse them with the spider's method parse_item
        Rule(SgmlLinkExtractor(allow='2016\/11\/01\/.*\/'), callback='parse_item')
    )

    def parse_item(self, response):
        # log here is for checking. Check if the parse_item really executed
        self.log('item page url: %s' % response.url)
        
        # extract title, author, content and key word from the extracted website data
        title = response.selector.xpath('//title/text()').extract()
        author = response.css('div.byline a::text').extract_first()

        raw_content = response.css('.article-entry').extract()[0]
        content = self.extract_content(raw_content)

        raw_key_words = response.selector.xpath('//script[@type="application/ld+json"]/text()').extract_first()
        key_words = self.extract_keywords(raw_key_words)

        item = TechcrunchItem()
        item['title'] = title
        item['author'] = author
        item['content'] = content
        item['key_words'] = key_words
        self.log('name of this item is: {}'.format(title))
        return item

    def extract_content(self, raw_content):
        # filter out unnecessary header and tails
        begin = '<!-- Begin: Wordpress Article Content -->'
        article_begin_index = raw_content.find(begin) + len(begin)
        end = '<!-- End: Wordpress Article Content -->'
        article_end_index = raw_content.find(end)
        content = raw_content[article_begin_index + 1:article_end_index]
        return content

    def extract_keywords(self, raw_key_words):
        # key words is in the script. Extract the script and get the keywords accordingly
        raw_key_words = raw_key_words.strip()
        word_dict = ujson.loads(raw_key_words)
        keyword_list = word_dict.get('keywords', [])
        return ', '.join(keyword_list) if keyword_list else ''


If running the code above gives you an implementation error, try change the Python path first:

```export PYTHONPATH=/Library/Python/2.7/site-packages:$PYTHONPATH```

## Parse crawled data

The CrawlSpider subclass mainly implemented functions to initiate and execute crawling process. Rules defines the allowed **extracting** crawling links and the allowed **parsing** crawling links. Callback for the parsing rule will invoke the parse_item method for you to extract data for every required sub-urls.

There are two ways for you to parse the website data. Here for better explaning the two methods, we have provided a mixed implementation. XPath expression is a powerful tool to decode and extract information from XML and HTML like files. XPath expressions offer more power because besides navigating the structure, it can also look at the content. Using XPath, you’re able to select things like: select the link that contains the text “Next Page”. This makes XPath very fitting to the task of scraping. CSS is another structure that perform similar tasks with XPath. If you look at the implementation, you'll find that CSS actually convert

## Store crawled data

Scrapy provides with a very easy way to store data. Using Item Object, user can define the storing attributes entity conveniently. The Object can guarantee that you store the same type of data in a consistent way. The attributes of feeding news for TechCrunch are defined in items.py:

In [ ]:
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# http://doc.scrapy.org/en/latest/topics/items.html

import scrapy
from scrapy.item import Item, Field


class TechcrunchItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = Field()
    author = Field()
    content = Field()
    key_words = Field()


If you run ```scrapy crawl newsFeed -o items.json -t json```, the result of items will be eventually stored in a file named items.json in spider folder.

## Checkpoint Summary

Till now, we have implemeted a crawler to extract TechCrunch news for a specific time range using Scrapy. The crawler have some basic functions including parsing and storing customized data, crawling data based on pagination and time range, and error handling. Next we are going to explore some advanced use of Scrapy.

## Page Limitation
You may notice that the previous crawler we implemented fetched every url and searched for random depth of the pages. In reality, when it referes to popular or complex website, crawling all possible urls is time-consuming and impractical. Based on different requirements, sometimes we only need the latest information, or information on certain depth of url search.

One of the easiest way of doing page limitation is to find patterns to url. Take TechCrunch for example, the url based on the page is like ```https://techcrunch.com/startups/page/2/```. So simply changing the rules as follows could meet the requirement.
```python
rules = (                                                                    
    rule(sgmllinkextractor(allow=('startups\/page\/[1-8]',))),               
    # extract links matching 'item.php' and parse them with the spider's method parse_item
    rule(sgmllinkextractor(allow='2016\/11\/01\/.*\/'), callback='parse_item')
)
```



However, sometimes the url does not imply page information, or the requirement asks you to only crawl 5 depth of the url, for example. Scrapy does not provide direct extension for that. But you can create a Scrapy middleware to hooks into Scrapy’s request/response processing.

First of all, we will uncomment and modify the parameters in setting.py to configure the depth limit and activate a downloader middleware component.

In [ ]:
SPIDER_MIDDLEWARES = {
    'myproject.middlewares.DomainDepthMiddleware': 900,
    'scrapy.contrib.spidermiddleware.depth.DepthMiddleware': None                  
}

Next we will implement the download middleware. But before that, we need to install some libraries. **tldextract** is a library for you to extract domain from url.

In [ ]:
# here is an example
import tldextract
url = 'https://techcrunch.com/2016/10/27/does-gillette-know-your-name/'
tldextract.extract(url).registered_domain

The basic steps to implement the middelware is: 
1) define settings:
DOMAIN_DEPTHS would be a dictionary with depth limits per domain
DEPTH_LIMIT setting we'll leave as a default one in case a domain is not configured

In [ ]:
DOMAIN_DEPTHS = {'techcrunch': 5}
DEPTH_LIMIT = 3

2) fetch the maximum depth of the current request, increment the depth value in its meta dictionary, and then check if it exceed the maximum depth of the url or the default depth if there's no maximum depth.

In [ ]:
from scrapy import log
from scrapy.http import Request
import tldextract


class DomainDepthMiddleware(object):
    def __init__(self, domain_depths, default_depth):
        self.domain_depths = domain_depths
        self.default_depth = default_depth

    @classmethod
    def from_crawler(cls, crawler):
        settings = crawler.settings
        domain_depths = settings.getdict('DOMAIN_DEPTHS', default={})
        default_depth = settings.getint('DEPTH_LIMIT', 5)

        return cls(domain_depths, default_depth)

    def process_spider_output(self, response, result, spider):
        def _filter(request):
            if isinstance(request, Request):
                # get max depth per domain
                domain = tldextract.extract(request.url).registered_domain
                maxdepth = self.domain_depths.get(domain, self.default_depth)

                depth = response.meta.get('depth', 0) + 1
                request.meta['depth'] = depth

                if maxdepth and depth > maxdepth:
                    log.msg(format="Ignoring link (depth > %(maxdepth)d): %(requrl)s ",
                            level=log.DEBUG, spider=spider,
                            maxdepth=maxdepth, requrl=request.url)
                    return False
            return True

        return (r for r in result or () if _filter(r))

## Email Notification
One of the frequently used functionalities of crawler is to give notifications. Users would get email notified regularly based on the rules for specific information. In the following session, we will show you how to build a crawler with timely notification.

Scrapy provides with convenient email notification using MailSender library. The basic use of MailSender is as simple as follows:

In [ ]:
from scrapy.mail import MailSender
mailer = MailSender()
mailer.send(to=["someone@example.com"], subject="Some subject", body="Some body", cc=["another@example.com"])

Note that the to_email and cc_email should be within a list, otherwise this would not work. From_mail is default to 'scrapy@localhost'.

Sometimes gmail would need your authentication, sending mails from personal or specific email addresses may need your mail box information. In that way, implementing a custmized send_mail function using Python library **smtplib** may be more convenient.

In [ ]:
import smtplib
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText

def send_mail(message, title):
    print "Sending mail..........."
    gmailUser = 'mail_you_send_from@gmail.com'
    gmailPassword = 'password'
    recipient = 'mail_to_send_to'

    msg = MIMEMultipart()
    msg['From'] = gmailUser
    msg['To'] = recipient
    msg['Subject'] = title
    msg.attach(MIMEText(message))

    mailServer = smtplib.SMTP('smtp.gmail.com', 587)
    mailServer.ehlo()
    mailServer.starttls()
    mailServer.ehlo()
    mailServer.login(gmailUser, gmailPassword)
    mailServer.sendmail(gmailUser, recipient, msg.as_string())
    mailServer.close()
    print "...Mail sent!"
send_mail("Testing from Scrapy tutorial", "Testing Scrapy")

By giving the email address, password and recipient address, we could send the email through SMTP. 

```if it says '...Mail sent!', then it means that the email has been sent successfully.
```

Now let's integrated the email component and the crawler together.

We want to find the latest news about **Instagram**, and get notified if such news exist. One way of doing that is to tell the crawler to filter from the key words. If there's **instagram** in the key words, customize the url link and key words as message content, and key words as title, to send the email out.

In [ ]:
    def parse_item(self, response):
        # log here is for checking. Check if the parse_item really executed
        self.log('item page url: %s' % response.url)

        # extract title, author, content and key word from the extracted website data
        title = response.selector.xpath('//title/text()').extract()
        author = response.css('div.byline a::text').extract_first()

        raw_content = response.css('.article-entry').extract()[0]
        content = self.extract_content(raw_content)

        raw_key_words = response.selector.xpath('//script[@type="application/ld+json"]/text()').extract_first()
        key_words = self.extract_keywords(raw_key_words)

        item = TechcrunchItem()
        item['title'] = title
        item['author'] = author
        item['content'] = content
        item['key_words'] = key_words
        if 'instagram' in key_words:
            messge_content = 'Key Words: {}'.format(key_words)
            messge_content += '\n' + 'url link: {}'.format(response.url)
            send_mail(messge_content, key_words)
        self.log('name of this item is: {}'.format(title))
        return item

## Make the crawler smarter
Here comes our final step to make the crawler more useful! How about getting a crawler which can send you emails with desired information every day (timely)?
Python provides with a useful package named CronTab, which can help you run your Python script in a timely manner. The template to write cron job command is as follows:
```
* * * * * command to be executed
- - - - -
| | | | |
| | | | ----- Day of week (0 - 7) (Sunday=0 or 7)
| | | ------- Month (1 - 12)
| | --------- Day of month (1 - 31)
| ----------- Hour (0 - 23)
------------- Minute (0 - 59)
```

So now we can create a con job script to execute commands. Be sure to install CronTab first.

In [ ]:
# the cron job task equals to:
# 0 0 * * * python path_to_news_spider.py
from crontab import CronTab
#init cron
cron = CronTab()

#add new cron job
job = cron.new(command='python path_to_news_spider.py')

#job settings
job.hour.every(24)

Command is a normal way to run python script, ```job.hour.every(24)``` equals cron job command ```0 0 * * * command to execute```. Changing the command according to your news_spider.py directory and run the script. Use ```crontab -l``` in terminal to see the list of current running script, and ```crontab -e``` to edit the cron jobs.

## Summary
Until now, we have introduced how Scrapy works, and have covered most of the basic use of Scrapy framework, and some interesting tasks built on top of it. There are a lot of interesting advanced topics that are not covered by this tutorial can be found at https://doc.scrapy.org/en/1.2/. Feel free to explore it!